In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,default_data_collator
import torch
from peft import PeftModel
from transformers import GenerationConfig
from transformers.generation.utils import LogitsProcessorList
from transformers.generation.logits_process import LogitsProcessor
from datasets import load_dataset
from torch.utils.data import DataLoader
import os

abs_path = "/data/home/chenpz/git_clone_project"
model_path = f"{abs_path}/All_base_model/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots/d3aa29f914761e8ea0298051fbaf8dd173e94db5"
data_path = f"/data/home/chenpz/git_clone_project/nlpData/mix_data_of_alpaca/FFP_all.json"
# adpter_path = f"/data/home/chenpz/git_clone_project/LLaMA-Factory/saves/llama3-8b-anli_r3_train_kcg_addHighPPL_first5000_data_gas=5_lr=1e-4/checkpoint-60"
output_file = '/data/home/chenpz/git_clone_project/jupyter_notebook_test/output/mixed_alpaca_answerPPL_FFP_all.jsonl'

output_dir = os.path.dirname(output_file)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


device = 'cuda:1'
prompt = \
'''<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{instruction} {input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{output}<|eot_id|>'''



# nf4_config = BitsAndBytesConfig(
#    load_in_4bit=True,
#    bnb_4bit_quant_type="nf4",
#    bnb_4bit_use_double_quant=True,
#    bnb_4bit_compute_dtype= None
# )


model = AutoModelForCausalLM.from_pretrained(model_path, 
                                             torch_dtype=torch.bfloat16,
                                            #  quantization_config=nf4_config,
                                             device_map = device)

# model = PeftModel.from_pretrained(
#                     model, adpter_path, is_trainable=False
#                     )




/data/home/chenpz/.conda/envs/sft/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.40it/s]


In [2]:
tokenizer = AutoTokenizer.from_pretrained(model_path,use_fast=False,split_special_tokens=False,
                                          padding_side="left",
                                          **{'trust_remote_code': True, 'cache_dir': None, 'revision': 'main', 'use_auth_token': None})
tokenizer.pad_token = '<|eot_id|>'

dataset = load_dataset("json", data_files= data_path)

IGNORE_INDEX = -100

question_prompt = '''<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{instruction}{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'''

answer_prompt = '''{output}<|eot_id|>'''


def preprocess_Answer_data(examples) :
        model_inputs = {"input_ids": [], "attention_mask": [], "labels": []}
        max_length = 400
        for  output in examples['output']:

            response= answer_prompt.format(output = output)
            
            input_ids, labels = [], []
            target_ids = tokenizer(response)['input_ids']
            input_ids +=  target_ids

            input_ids = input_ids[:max_length]

            labels += target_ids
            labels = labels[:max_length]
            
            model_inputs["input_ids"].append(input_ids)
            model_inputs["attention_mask"].append([1] * len(input_ids))
            model_inputs["labels"].append(labels)

        # Padding: Ensure that all input sequences have the same length
        for i in range(len(model_inputs["input_ids"])):
            padding_length = max_length - len(model_inputs["input_ids"][i])
            
            # Left padding input_ids and attention_mask
            model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * padding_length + model_inputs["input_ids"][i]
            model_inputs["attention_mask"][i] = [0] * padding_length + model_inputs["attention_mask"][i]

            # Left padding labels with IGNORE_INDEX
            model_inputs["labels"][i] = [IGNORE_INDEX] * padding_length + model_inputs["labels"][i]
        
        return model_inputs


dataset2 = dataset.map(preprocess_Answer_data,batched=True,remove_columns=['output', 'input', 'instruction'],num_proc=16)
print(tokenizer.decode(dataset2['train'][0]['input_ids']))

eval_dataloader = DataLoader(dataset2['train'],batch_size=6, pin_memory=True,collate_fn=default_data_collator,shuffle=False)


<|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|>

In [3]:
from tqdm import tqdm
model.eval()
loss_list =[]
loss_fn = torch.nn.CrossEntropyLoss(
    reduction='none'
    ) # 不要平均，保留每个 token 的 loss
with open(output_file, 'w') as file:  # Open the file in write mode before the loop
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], labels=batch['labels'])

            logits = outputs.logits
            shift_logits = logits[..., :-1, :].contiguous()  # 移动 logits 以对齐预测
            shift_labels = batch['labels'][:, 1:].contiguous()  # 移动 labels
            
            # 计算每个 token 的 loss
            loss_per_token = loss_fn(shift_logits.view(-1, model.config.vocab_size), shift_labels.view(-1))
            # reshape 回到 batch 大小
            loss_per_token = loss_per_token.view(shift_labels.size())
            

            means = loss_per_token.sum(dim=-1) / (loss_per_token != 0).sum(dim=-1)
            means = list( means.cpu().numpy() )

        # break
            # print(loss_per_sample)
            for i, loss in enumerate(means):
                # print(loss)
                # break
                sample_id = step * eval_dataloader.batch_size + i  # 用于区分每个样本的索引
                add_item = {'id':sample_id,'loss':loss}
                file.write('%s\n' % add_item)

100%|██████████| 20294/20294 [1:37:39<00:00,  3.46it/s]


In [ ]:
means.cpu().numpy()